# Whole brain data processing -- demix
### This is run as some denoised results are available

In [1]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from cellProcessing import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def force_bkill_dask():
    try:
        get_ipython().run_cell_magic('bash', '', 'bkill -q normal 0\n')
    except:
        get_ipython().run_cell_magic('bash', '', 'bjobs\n')

## Set up environments

* Install `fish_proc` from github https://github.com/zqwei/fish_processing

In [3]:
## STEP 1: specify data and save path
dir_root = '/groups/ahrens/ahrenslab/jing/giving_up/20190219/fish2/7dpf-huc_gcamp7ff_gfap_rgeco-GU_slow_fwd-fish02-exp02_20190219_174013/im'
# save_root = '/nrs/ahrens/jing/20190219/fish2/7dpf-huc_gcamp7ff_gfap_rgeco-GU_slow_fwd-fish02-exp02_20190219_174013/processed'

### Set save folder to Ziqiang's folder

In [4]:
save_root = './processed'
if not os.path.exists(save_root):
    os.makedirs(save_root)

## Demix
This will generate the follow files at `save_root` folder
* `demix_rlt` for each block

### Check denoised signal

In [5]:
dt = 3

In [8]:
from fish_proc.utils.snr import correlation_pnr
import zarr
nz = 0
Y_svd = zarr.open(f'{save_root}/masked_local_pca_data_{nz}.zarr', 'r')

In [10]:
_ = Y_svd[:, :, ::dt]

In [14]:
np.isnan(_).sum()

66530688

In [ ]:
Cn, _ = correlation_pnr(, skip_pnr=True)

In [ ]:
## mask parameters
intensity_percentile = 50  
dt = 3  # time downsample for cell segmentation
mask_brain(save_root, percentile=intensity_percentile, dt=dt, numCores=20, is_skip_snr=True, save_masked_data=True)

In [ ]:
numCores = 200
demix_cells(save_root, numCores = numCores)

### Check demix results

In [ ]:
block_id = (0, 2, 0, 0)
check_demix_cells(save_root, block_id, nsplit=nsplit, plot_global=True)

## Compute df/f
One of three choices
* df is from NMF components (where background is not included) -- preferred
* df is on raw data based on the recomputation of cell F using NMF weights
* df/f on pixels without using NMF weights

### df/f on pixels

In [ ]:
numCores = 70
dff = compute_cell_dff_pixels(dir_root, save_root, 
                              numCores=numCores, 
                              window=baseline_window, 
                              percentile=baseline_percentile)

### df/f on raw cell data

In [ ]:
compute_cell_dff_raw(dir_root, save_root, 
                     numCores=numCores, 
                     window=baseline_window, 
                     percentile=baseline_percentile, 
                     nsplit=nsplit)

### df/f on denoised cell data

In [ ]:
compute_cell_dff_NMF(dir_root, save_root, 
                     numCores=numCores, 
                     window=baseline_window, 
                     percentile=baseline_percentile, 
                     nsplit=nsplit, dt=dt)